In [25]:
import pandas as pd
from bs4 import BeautifulSoup
import os
import requests

# Load the input Excel file
input_file = "input.xlsx"
df = pd.read_csv(input_file)
df

URL_ID                                                URL
0      123.0  https://insights.blackcoffer.com/rise-of-telem...
1      321.0  https://insights.blackcoffer.com/rise-of-e-hea...
2     2345.0  https://insights.blackcoffer.com/rise-of-e-hea...
3     4321.0  https://insights.blackcoffer.com/rise-of-telem...
4      432.0  https://insights.blackcoffer.com/rise-of-telem...
..       ...                                                ...
109  50921.0  https://insights.blackcoffer.com/coronavirus-i...
110  51382.8  https://insights.blackcoffer.com/coronavirus-i...
111  51844.6  https://insights.blackcoffer.com/what-are-the-...
112  52306.4  https://insights.blackcoffer.com/marketing-dri...
113  52768.2  https://insights.blackcoffer.com/continued-dem...

[114 rows x 2 columns]

In [26]:
# Ensure the output directory exists
output_dir = "extracted_articles"
os.makedirs(output_dir, exist_ok=True)

In [27]:
# Function to extract the heading of the article
def extract_article_heading(html_content):
    
    soup = BeautifulSoup(html_content, 'html.parser')
        
    # Find the <h1> tag within the "proftags" div
    article_heading = soup.find("h1")
    
    # Process or return the heading as needed
    if article_heading:
        heading_text = article_heading.get_text()
        return heading_text



# Function to extract the entire article text
def extract_article_text(html_content):
    
    soup = BeautifulSoup(html_content, 'html.parser')
        
    # Find the div with class "td-post-content tagdiv-type"
    proftags = soup.find("div", {"class": "td-post-content tagdiv-type" })

    # Check if the div was found
    if proftags is not None:
        # The div was found; proceed to extract the data.

        # Find all <p> tags within the "proftags" div, excluding <pre> tags
        article_paragraphs = proftags.find_all("p", recursive=False)

        # Join all paragraphs into a single text
        article_text = '\n'.join(paragraph.get_text() for paragraph in article_paragraphs)

        return article_text
    else:
        return ""  # Return an empty string or handle the case where the div is not found

In [28]:
# Loop through each row in the Excel file
for index, row in df.iterrows():
    url_id = row['URL_ID']
    url = row['URL']

    # Make a request to the URL
    response = requests.get(url)

    if response.status_code == 200:
        article_html = response.text

        article_text = extract_article_text(article_html)
        if article_text:
            with open(os.path.join(output_dir, f"{url_id}.txt"), "w", encoding="utf-8") as text_file:
                # Write the heading to the text file
                article_heading = extract_article_heading(article_html)
                if article_heading:
                    text_file.write(article_heading)

                # Leave a line and then write the article text
                text_file.write("\n")
                text_file.write(article_text)
        else:
            print(f"Failed to retrieve HTML content for URL_ID {url_id}")

print("Article text extraction and saving completed.")

Failed to retrieve HTML content for URL_ID 2345.0
Failed to retrieve HTML content for URL_ID 4279.2
Failed to retrieve HTML content for URL_ID 8435.4
Failed to retrieve HTML content for URL_ID 14900.6
Failed to retrieve HTML content for URL_ID 33372.6
Failed to retrieve HTML content for URL_ID 33834.4
Failed to retrieve HTML content for URL_ID 37528.8
Failed to retrieve HTML content for URL_ID 40761.4
Failed to retrieve HTML content for URL_ID 41223.2
Failed to retrieve HTML content for URL_ID 43070.4
Article text extraction and saving completed.
